In [177]:
import pandas as pd
import yfinance as yf
from yahoo_fin.stock_info import *
import numpy as np

In [178]:
class Stock: 

    def __init__(self, ticker, period="1mo"):
        self.ticker = ticker
        self.period = period

# Gets the adjusted close of a ticker per a certain period
    def get_prices(self):
        price_data = get_data(self.ticker, start_date = "20/03/2017", index_as_date = True, interval=self.period)
        adjclose = price_data.loc[:, 'adjclose']
        return adjclose.tolist()


    # Gets a list of the returns per period in percentage form
    def get_period_returns(self):
        adjclose_list = self.get_prices()

    # Loops through the adjusted closes and sums the ubiased daily return divided by the length of the list
        period_returns = []

        for i in range(len(adjclose_list) - 1):
            daily_return = np.log(adjclose_list[i+1]/adjclose_list[i]) * 100
            if np.isnan(daily_return) == False:
                period_returns.append(daily_return)
        
        return period_returns

    # Gets the mean return of the stock per-period
    def get_mean_return(self):
        period_returns = self.get_period_returns()

        mean_return = sum(period_returns)/len(period_returns)
        return mean_return

    # Gets the standard deviation of a certain stock
    def get_risk(self):

        period_returns = self.get_period_returns()
        return np.std(period_returns, dtype=np.float64)


doge = Stock('luna-usd')
print(doge.get_risk())





91.10925526727114


# Minimum variance portoflio
The following module calculates the weightings of two stocks that give the the least risk

Disclaimer: Both stocks must have existed for the same amount of time

In [179]:
class Portfolio: 
    def __init__(self, ticker_1, ticker_2, period = "1mo"):
        self.stock_1 = Stock(ticker_1, period)    
        self.stock_2 = Stock(ticker_2, period)
    
    # Gets the covariance of the two stocks
    def get_cov(self):
        returns1 = self.stock_1.get_period_returns()
        returns2 = self.stock_2.get_period_returns()

        cov_matrix = np.cov(returns1,returns2)

        cov = cov_matrix[0,1]
        return cov
        

    def get_corr(self):
        corr = self.get_cov()/(self.stock_1.get_risk() * self.stock_2.get_risk())
        return corr

    def get_weights(self):
        std1 = self.stock_1.get_risk()
        std2 = self.stock_2.get_risk()

        numerator = (std2**2) - (self.get_corr() * std1 * std2)
        denominator = (std1**2) + (std2**2) - (2 * self.get_corr() * std1 * std2)

        stock_1_weight = numerator/denominator
        stock_2_weight = 1 - stock_1_weight

        return (stock_1_weight, stock_2_weight)


portfolio = Portfolio('btc-usd', 'msft')
print(portfolio.get_weights())

(-0.016079299901901874, 1.016079299901902)
